<a href="https://colab.research.google.com/github/CaQtiml/OSUgradingprogram_2190102final/blob/main/OSUgradingprogram_FlaskBackend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sivakorn Lerttripinyo 6338212621

In [ ]:
!pip install pymongo[srv]

     |████████████████████████████████| 241 kB 29.4 MB/s 


In [ ]:
import pymongo
import urllib

In [ ]:
client = pymongo.MongoClient("mongodb+srv://<conceal>/myFirstDatabase?retryWrites=true&w=majority")

In [ ]:
! pip install flask_ngrok
! pip install flask_cors
! pip install pymongo[srv]

In [ ]:
from flask import Flask,request,flash, request, redirect, url_for, Response,jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin
import pymongo
import json
import os
from werkzeug.utils import secure_filename
from werkzeug.wrappers import Response
from collections import OrderedDict
import pandas as pd
import numpy as np

In [ ]:
app = Flask(__name__)
app.config['JSON_SORT_KEYS'] = False
CORS(app, support_credentials=True)
run_with_ngrok(app)

In [ ]:
@app.route('/insert', methods=['POST', 'GET'])
def insert_db():
    db = client.project.score
    if request.method == "GET":
        id = request.args.get("id")
        mt = int(request.args.get("mt"))
        fn = int(request.args.get("fn"))
        asgm = int(request.args.get("asgm"))
    elif request.method == "POST":
        body = request.get_json()
        id = body["id"]
        mt = body["mt"]
        fn = body["fn"]
        asgm = body["asgm"]
    sum = mt + fn + asgm
    r = db.insert_one({'id':id,
                'mt':mt,
                'fn':fn,
                'asgm':asgm,
                'total':sum})
    return jsonify(success=True)
    

def geval(score: int, criteria: list) -> str:
    if score >= criteria[0] :
        return "O"
    elif score >= criteria[1] :
        return "S"
    else:
        return "U"

def getCriteria() -> list:
    db_cri = client.project.criteria
    temp = db_cri.find_one({})
    criteria = [int(temp["OC"]), int(temp["SC"])]
    return criteria

@app.route('/getbyrange')
def query_range():
    db = client.project.score
    criteria = getCriteria()
    minScore = int(request.args.get("min"))
    maxScore = int(request.args.get("max"))
    rows = [{ 
                "id": row["id"], 
                "midterm": row["mt"],
                "final": row["fn"],
                "assignment": row["asgm"],
                "totalScore": row["total"],
                "grade": geval(row["total"], criteria) } \
        for row in db.find({"total":{"$gte":minScore, "$lte":maxScore}}).sort("total", pymongo.ASCENDING)]
    ret = {"data":rows, "criteria":criteria}
    return jsonify(ret)

@app.route('/getallgrades')
def getallgrades() -> Response:
    db = client.project.score
    db_cri = client.project.criteria
    OC = int(request.args.get("oc"))
    SC = int(request.args.get("sc"))
    temp = db_cri.find_one({})
    if not(temp["OC"]==OC and temp["SC"]==SC):
        new = {"$set":{"OC":OC, "SC":SC}}
        db_cri.update_one(temp, new)
    criteria = [OC,SC]
    queries = db.find({}).sort("total", pymongo.DESCENDING)
    data = []
    dt = []
    for row in queries:
        data.append({"id": row["id"], "score":row["total"], "grade":geval(row["total"],criteria)})
        parscore = row["total"]
        dt.append(int(parscore))
    dt = np.array(dt)
    cnt = [np.count_nonzero(dt >= OC),
           np.count_nonzero(np.logical_and(SC <= dt,dt < OC)),
           np.count_nonzero(dt < SC)]
    average = round(np.mean(dt),2)
    std = round(np.std(dt,ddof=1),2)
    output = {"data": data, "cnt":cnt, "avg":average, "std":std}
    return jsonify(output)

@app.route('/getgroup')
def getgroup() -> Response:
    db = client.project.score
    criteria = getCriteria()
    ids = request.args.get("ids")
    idsArr = ids.split(" ")
    idsArr = [element.strip() for element in idsArr]
    print(idsArr)
    data = db.find({"id":{"$in":idsArr}})
    rows = [{
                "id": row["id"], 
                "midterm": row["mt"],
                "final": row["fn"],
                "assignment": row["asgm"],
                "totalScore": row["total"],
                "grade": geval(row["total"], criteria)    
            } for row in data]
    ret = {"data":rows, "criteria":criteria}
    return jsonify(ret)

app.run()

/insert?id=111&mt=10&fn=10&asgm=10

/getbyrange?min=10&max=40

/getallgrades?oc=25&sc=19

/getgroup?ids=81112 65